In [1]:
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import time
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
from threading import Thread
import queue

In [2]:
# email address required to access GenBank
Entrez.email = "megzlives@gmail.com"

In [3]:
# function to retrieve genomic data
def retrieve_genomic_data(accession):
    print(f"Attempting to retrieve data for {accession}")
    for attempt in range(3):  # Try to fetch the data up to 3 times
        try:
            handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gbwithparts", retmode="text")
            print(f"Downloading data for {accession}...")
            # Use SeqIO.parse() to read multi-sequence GenBank files
            records = list(SeqIO.parse(handle, "genbank"))
            handle.close()
            print(f"Successfully retrieved data for {accession}")
            return records  # This will be a list of records
        except Exception as e:
            print(f"An error occurred while fetching {accession}: {e}")
            print("Attempting to retry...")
            time.sleep(2)  # Wait for 2 seconds before retrying
    print(f"Failed to retrieve data for {accession} after multiple attempts.")
    return None

In [ ]:
# sequentially retrieve genomic data for each accession number of Dienoccocus radiodurans
#d_radiodurans_sequences = {}
#accession_numbers = [
   # "NZ_CP038663.1",  # Chromosome I
  #  "NZ_CP038664.1",  # Chromosome II
  #  "NZ_CP038666.1",  # Plasmid pCP1
  #  "NZ_CP038665.1",  # Plasmid pMP1
#]

#for accession in accession_numbers:
  #  records = retrieve_genomic_data(accession)
  #  if records:
  #      d_radiodurans_sequences[accession] = records
  #      print(f"Records for {accession} added to the database.")
  #  else:
  #      print(f"Could not retrieve records for {accession}.")

In [4]:
# Define a dictionary to hold the accession numbers for each organism and their components
accession_numbers = {
    "Deinococcus radiodurans": ["NZ_CP038663.1", "NZ_CP038664.1", "NZ_CP038666.1", "NZ_CP038665.1"],
    "Thermus thermophilus": ["NC_006461.1", "NC_006463.1", "NC_006462.1"], 
    "Deinococcus radiophilus": ["NZ_CP086380.1", "NZ_CP086381.1", "NZ_CP086382.1", "NZ_CP086384.1", "NZ_CP086383.1", "NZ_CP086385.1", "NZ_CP086386.1"],
    "Deinococcus proteolyticus": ["NC_015161.1", "NC_015169.1", "NC_015162.1", "NC_015170.1", "NC_015163.1"],
    "Deinococcus geothermalis": ["NC_008025.1", "NC_008010.2", "NC_009939.1"]
}

In [5]:
# Retrieve genomic data for each organism
all_organism_sequences = {}
for organism, accessions in accession_numbers.items():
    all_organism_sequences[organism] = []
    for accession in accessions:
        records = retrieve_genomic_data(accession)
        if records:
            all_organism_sequences[organism].extend(records)
            print(f"Records for {accession} ({organism}) added to the database.")
        else:
            print(f"Could not retrieve records for {accession} ({organism}).")

Attempting to retrieve data for NZ_CP038663.1
Successfully retrieved data for NZ_CP038663.1
Records for NZ_CP038663.1 (Deinococcus radiodurans) added to the database.
Attempting to retrieve data for NZ_CP038664.1
Successfully retrieved data for NZ_CP038664.1
Records for NZ_CP038664.1 (Deinococcus radiodurans) added to the database.
Attempting to retrieve data for NZ_CP038666.1
Successfully retrieved data for NZ_CP038666.1
Records for NZ_CP038666.1 (Deinococcus radiodurans) added to the database.
Attempting to retrieve data for NZ_CP038665.1
Successfully retrieved data for NZ_CP038665.1
Records for NZ_CP038665.1 (Deinococcus radiodurans) added to the database.
Attempting to retrieve data for NC_006461.1
Successfully retrieved data for NC_006461.1
Records for NC_006461.1 (Thermus thermophilus) added to the database.
Attempting to retrieve data for NC_006463.1
Successfully retrieved data for NC_006463.1
Records for NC_006463.1 (Thermus thermophilus) added to the database.
Attempting to re

In [ ]:
# function to perform BLAST analysis for a gene sequence from D. radiodurans against an organism
#def perform_blast(sequence, database="nt", organism="Thermus thermophilus"):
   # try:
    #    result_handle = NCBIWWW.qblast("blastn", database, sequence, entrez_query=f'organism="{organism}"')
    #    return result_handle
  #  except Exception as e:
    #    print(f"Error performing BLAST: {str(e)}")
     #   return None

In [6]:
# times out if BLAST search takes too long. time is in seconds 
# change organism to the desired species for the BLAST search
def perform_blast(sequence, database="nt", organism="Thermus thermophilus", timeout=900):
    def blast_thread(queue, sequence, database, organism):
        try:
            result_handle = NCBIWWW.qblast("blastn", database, sequence, entrez_query=f'organism="{organism}"')
            queue.put(result_handle)
        except Exception as e:
            print(f"Error performing BLAST: {str(e)}")
            queue.put(None)

    q = queue.Queue()
    blast_thread = Thread(target=blast_thread, args=(q, sequence, database, organism))
    blast_thread.start()
    blast_thread.join(timeout)

    if blast_thread.is_alive():
        print("BLAST search timed out")
        return None
    else:
        return q.get()

In [7]:
# function for extracting gene sequence 
# change search_type to gene to search for gene name or locus_tag to search by locus tag
# change organism to the organism you want to search for target genes in
def extract_gene_sequence(search_term, search_type='locus_tag', organism='Deinococcus radiodurans'):
    with open("gene_and_locus_tags.txt", "w") as file:  # Open a text file for writing
        found = False  # Flag to check if the gene is found
        for record in all_organism_sequences[organism]:
            for feature in record.features:
                if feature.type in ["gene", "CDS"]:
                    # Extract gene name, locus tag, and gene ID
                    gene_name = feature.qualifiers.get("gene", ["No gene name"])[0]
                    locus_tag = feature.qualifiers.get("locus_tag", ["No locus tag"])[0]
                    gene_id_list = feature.qualifiers.get("db_xref", [])
                    gene_id = next((id_part.split(":")[1] for id_part in gene_id_list if id_part.startswith("GeneID:")), "No Gene ID")
                    file.write(f"{gene_name}\t{locus_tag}\t{gene_id}\n")  # Write to file
                    if search_type in feature.qualifiers:
                        # This can be a list, so check if the search term is in the list
                        if search_term in feature.qualifiers[search_type]:
                            print(f"Found {search_type} {search_term} in {organism}, Record ID: {record.id}")
                            found = True
                            gene_sequence = feature.extract(record.seq)
                            return gene_sequence
            if not found:
                print(f"Gene with {search_type} {search_term} not found in record {record.id}.")
    if not found:
        print(f"Search completed. Gene with {search_type} {search_term} not found in any of the GenBank records.")
    return None


In [8]:
# function to parse BLAST results
def parse_blast_results(blast_result_handle):
    blast_records = NCBIXML.read(blast_result_handle)
    results = []
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
            result = {
                'title': alignment.title,
                'score': hsp.score,
                'e_value': hsp.expect,
                'identities': hsp.identities,
                'align_length': hsp.align_length
            }
            results.append(result)
    return results

In [9]:
# function to display BLAST results
def display_blast_results(blast_results):
    for gene, results in blast_results.items():
        print(f"Results for {gene}:")
        for result in results:
            print(f"Title: {result['title']}")
            print(f"Score: {result['score']}")
            print(f"E-value: {result['e_value']}")
            print(f"Identities: {result['identities']}")
            print(f"Alignment length: {result['align_length']}")
            print("\n")

In [10]:
# dictionary of target genes in D. radiodurans; includes gene name and locus tags
target_genes = {
    "PprA": "E5E91_RS15025",
    "RecA": "E5E91_RS11810",
    "DdrA": "E5E91_RS02140",
    "DdrB": "E5E91_RS00360",
    "DdrC": "E5E91_RS00015",
    "NADH-quinone oxidoreductase subunit N": "E5E91_RS07520",
    "LigD": None,  # Replace with the actual locus tag when found
    "Dps": "E5E91_RS11440",
    "Bcp": None,   # Replace with the actual locus tag when found
    "IrrE": "E5E91_RS00855",
    "Ssb": None,   # Replace with the actual locus tag when found
    "PolA": "E5E91_RS08585",
    "ThyA": "E5E91_RS13290",
    "PNPase": "E5E91_RS10905"
}

In [11]:
# dictionary to store BLAST results for each gene
blast_results = {}

In [12]:
# extract gene sequences based on locus tag
for gene, locus_tag in target_genes.items():
    if locus_tag:  # Make sure the locus tag is not None
        gene_sequence = extract_gene_sequence(locus_tag, search_type='locus_tag')
        if gene_sequence:
            # Placeholder for the code to proceed with BLAST or other processing
            print(f"Gene sequence found for {gene}: {gene_sequence[:10]}...")  # Show part of the gene sequence
        else:
            print(f"Sequence extraction failed for gene: {gene}")
    else:
        print(f"Locus tag not found for gene: {gene}")

Gene with locus_tag E5E91_RS15025 not found in record NZ_CP038663.1.
Found locus_tag E5E91_RS15025 in Deinococcus radiodurans, Record ID: NZ_CP038664.1
Gene sequence found for PprA: GTGCTACCCC...
Found locus_tag E5E91_RS11810 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence found for RecA: ATGAGCAAGG...
Found locus_tag E5E91_RS02140 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence found for DdrA: ATGAAGCTGA...
Found locus_tag E5E91_RS00360 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence found for DdrB: ATGTTGCAGA...
Found locus_tag E5E91_RS00015 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence found for DdrC: ATGAAGAACG...
Found locus_tag E5E91_RS07520 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence found for NADH-quinone oxidoreductase subunit N: ATGAATCTGG...
Locus tag not found for gene: LigD
Found locus_tag E5E91_RS11440 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Gene sequence foun

In [ ]:
# extract gene sequences based on gene name
#for gene_name in target_genes.keys():
    #gene_sequence = extract_gene_sequence(gene_name, search_type='gene')
    #if gene_sequence:
        # Proceed with BLAST, etc.
    #else:
        #print(f"Sequence extraction failed for gene: {gene_name}")

In [13]:
# Perform BLAST and parse results
for gene, locus_tag in target_genes.items():
    print(f"Processing gene: {gene}")  # Debugging print
    if locus_tag:  # Make sure the locus tag is not None
        gene_sequence = extract_gene_sequence(locus_tag, search_type='locus_tag')
        if gene_sequence:
            print(f"Performing BLAST search for {gene}...")
            result_handle = perform_blast(str(gene_sequence), organism="Thermus thermophilus")
            if result_handle:
                # Parse the BLAST result handle and store it in the blast_results dictionary
                blast_results[gene] = parse_blast_results(result_handle)
                print(f"BLAST search completed for {gene}")
            else:
                print(f"BLAST search failed for gene: {gene}")
        else:
            print(f"Sequence extraction failed for gene: {gene}")
    else:
        print(f"Locus tag not found for gene: {gene}")

Processing gene: PprA
Gene with locus_tag E5E91_RS15025 not found in record NZ_CP038663.1.
Found locus_tag E5E91_RS15025 in Deinococcus radiodurans, Record ID: NZ_CP038664.1
Performing BLAST search for PprA...
BLAST search completed for PprA
Processing gene: RecA
Found locus_tag E5E91_RS11810 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Performing BLAST search for RecA...
BLAST search completed for RecA
Processing gene: DdrA
Found locus_tag E5E91_RS02140 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Performing BLAST search for DdrA...
BLAST search completed for DdrA
Processing gene: DdrB
Found locus_tag E5E91_RS00360 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Performing BLAST search for DdrB...
BLAST search completed for DdrB
Processing gene: DdrC
Found locus_tag E5E91_RS00015 in Deinococcus radiodurans, Record ID: NZ_CP038663.1
Performing BLAST search for DdrC...
BLAST search completed for DdrC
Processing gene: NADH-quinone oxidoreductase subunit N
Found locus

In [14]:
# Function to save BLAST results to a CSV file (IE, there was too much data to just display lol)
def save_blast_results_to_csv(blast_results, file_name):
    rows = []
    for gene, results in blast_results.items():
        for result in results:
            row = {
                'gene': gene,
                'title': result['title'],
                'score': result['score'],
                'e_value': result['e_value'],
                'identities': result['identities'],
                'align_length': result['align_length']
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    df.to_csv(file_name, index=False)

# Call the function to save BLAST results
save_blast_results_to_csv(blast_results, "blast_results.csv")

In [ ]:
# optionally the BLAST results (for error testing mostly, RIP)
print("BLAST results keys:", blast_results.keys())
display_blast_results(blast_results)

In [ ]:
# TODO: statistics stuff...definitely needs editing 
# Read BLAST results from CSV file into a DataFrame
df = pd.read_csv("blast_results.csv")

# Data Validation: Filter out results based on criteria, e.g., e-value threshold
df_filtered = df[df['e_value'] < 0.05]  # example threshold...what should it actually be?

# Perform Statistical Tests
# t-test for comparing scores between two groups
group1 = df_filtered[df_filtered['gene'] == 'gene1']['score']
group2 = df_filtered[df_filtered['gene'] == 'gene2']['score']
t_stat, p_val = ttest_ind(group1, group2)
print(f"t-test: t-statistic = {t_stat}, p-value = {p_val}")

# Chi-Square Test
# data needs to be reformatted for a chi-square test
# is it worth it? Probably not. 

In [ ]:
# DATA VISUALIZATION
# Histogram of BLAST Scores
plt.hist(df_filtered['score'], bins=30)
plt.title('Distribution of BLAST Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

# Boxplot for Score Comparison
sns.boxplot(x='gene', y='score', data=df_filtered)
plt.title('BLAST Score Comparison by Gene')
plt.xlabel('Gene')
plt.ylabel('BLAST Score')
plt.show()